# Graph Breadth First Search
In this exercise, you'll see how to do a breadth first search on a graph. To start, let's create a graph class in Python.

In [44]:
class GraphNode(object):
    def __init__(self, val):
        self.value = val
        self.children = []
        
    def add_child(self,new_node):
        self.children.append(new_node)
    
    def remove_child(self,del_node):
        if del_node in self.children:
            self.children.remove(del_node)
            
    def __repr__(self):
        return self.value

class Graph(object):
    def __init__(self,node_list):
        self.nodes = node_list
        
    def add_edge(self,node1,node2):
        if(node1 in self.nodes and node2 in self.nodes):
            node1.add_child(node2)
            node2.add_child(node1)
            
    def remove_edge(self,node1,node2):
        if(node1 in self.nodes and node2 in self.nodes):
            node1.remove_child(node2)
            node2.remove_child(node1)

Now let's create the graph.

In [45]:
nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS,nodeH,nodeG,nodeP,nodeR,nodeA] ) 
graph1.add_edge(nodeG,nodeR)
graph1.add_edge(nodeA,nodeR)
graph1.add_edge(nodeA,nodeG)
graph1.add_edge(nodeR,nodeP)
graph1.add_edge(nodeH,nodeG)
graph1.add_edge(nodeH,nodeP)
graph1.add_edge(nodeS,nodeR)

## Implement BFS
Using what you know about BFS for trees and DFS for graphs, let's do BFS for graphs. Implement the `bfs_search` to return the `GraphNode` with the value `search_value` starting at the `root_node`.

In [46]:
from collections import deque

"""
Following solution (and Udacity's solution) is INEFFICIENT, for example, in the graph below, it will add nodeA 
multiple times to the queue, one as a neighbor of G, and then, as a neighbor of R (uncomment BFS-Traversal print 
statements to see this). A more efficient implementationtracks the visited nodes (nodes added to the queue) and not 
explored nodes (nodes popped from the queue), making sure that an already visited node is not added again in the queue. This implementation can be found in the next cell.
"""

def bfs_search(root_node, search_value):
    """ relatively inefficent solution, see next cell for more efficient implementation """
    queue = deque()
    queue.appendleft(root_node)
    
    explored = set()
#     print('--BFS-Traversal--')
    while len(queue) > 0:
        node = queue.pop() # assuming right is front, left is rear
#         print(node.value)
        if node.value == search_value:
            return node
        explored.add(node.value)
        
        for child in node.children:
            if child.value not in explored:
                queue.appendleft(child)

In [54]:
def bfs_search(root_node, search_value):
    """ better solution, each node visited exactly once """
    
    queue = deque()
    queue.appendleft(root_node)
    
    visited = set()
    visited.add(root_node.value)
    
#     print('--BFS-Traversal--')
    while len(queue) > 0:
        node = queue.pop() # assuming right is front, left is rear
#         print(node.value)
        if node.value == search_value:
            return node
        
        for child in node.children:
            if child.value not in visited:
                queue.appendleft(child)
                visited.add(child.value)

In [51]:
nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS,nodeH,nodeG,nodeP,nodeR,nodeA] ) 
graph1.add_edge(nodeG,nodeR)
graph1.add_edge(nodeA,nodeR)
graph1.add_edge(nodeA,nodeG)
graph1.add_edge(nodeR,nodeP)
graph1.add_edge(nodeA,nodeP)
graph1.add_edge(nodeH,nodeP)
graph1.add_edge(nodeS,nodeP)

print(bfs_search(nodeG, 'H'))

H


<span class="graffiti-highlight graffiti-id_fg1wpq1-id_g7fi7m5"><i></i><button>Hide Solution</button></span>

In [52]:
def bfs_search(root_node, search_value):
    """ inefficient solution, see comment at the topmost cell of this section """
    visited = []
    queue = [root_node]
    
    while len(queue) > 0:
        current_node = queue.pop(0)
        visited.append(current_node)

        if current_node.value == search_value:
            return current_node

        for child in current_node.children:
            if child not in visited:
                queue.append(child)

### Tests

In [53]:
assert nodeA == bfs_search(nodeS, 'A')
assert nodeS == bfs_search(nodeP, 'S')
assert nodeR == bfs_search(nodeH, 'R')

# Bipartition check 

Following is a learning exercise that I put in, and was not a part of original notebook. The problem is to check whether a given connected graph is bipartite or not. Solution based on understanding from: https://www.youtube.com/watch?v=r1-8p11fSPw&list=PLBF3763AF2E1C572F&index=26

Basically, the idea is that check for bipartition can be done in two ways:
1. Ensure that there is no odd cycle in graph, or equivalently 
2. While performing bfs, make sure that there is no edge within the same level (set of nodes at same distance from start node). But, in order to check that need to maintain level number of nodes (or shortest distance from root).

In [77]:
def bipartite_check(root_node):
    """ check whether a graph is bipartite, assuming graph is connected. In case it is not connected, we will
    need to multiple BFS and make sure each connected component is bipartite, for the complete graph to be bipartite."""
    
    queue = deque()
    queue.appendleft(root_node)
    
    node_to_level = {}
    node_to_level[root_node] = 0
    
    while len(queue) > 0:
        node = queue.pop() # assuming right is front, left is rear
        node_level = node_to_level[node]
        
        for child in node.children:
            if child not in node_to_level:
                queue.appendleft(child)
                node_to_level[child] = node_level+1
            elif node_to_level[child] == node_level: # a graph-edge found within the same bfs-level implying the graph is not bipartite (or there is an odd cycle in the graph)
                return False
    return True        

In [78]:
# example of non-bipartite graph, contains odd-cycle

nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS,nodeH,nodeG,nodeP,nodeR,nodeA] ) 
graph1.add_edge(nodeG,nodeR)
graph1.add_edge(nodeA,nodeR)
graph1.add_edge(nodeA,nodeG)
graph1.add_edge(nodeR,nodeP)
graph1.add_edge(nodeA,nodeP)
graph1.add_edge(nodeH,nodeP)
graph1.add_edge(nodeS,nodeP)
print(bipartite_check(nodeG))

False


In [79]:
# example of bipartite graph, does NOT contain an odd cycle

nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS,nodeH,nodeG,nodeP,nodeR,nodeA] ) 
graph1.add_edge(nodeG,nodeR)
graph1.add_edge(nodeA,nodeR)
graph1.add_edge(nodeA,nodeP)
graph1.add_edge(nodeP,nodeH)
graph1.add_edge(nodeH,nodeS)
print(bipartite_check(nodeG))

True
